In [11]:
import os
import cv2
import numpy as np
import random
import pandas as pd

In [2]:
def load_images_from_folder(input_folder, output_folder, target_size):
    images = []

    for filename in os.listdir(input_folder):
        img = cv2.imread(os.path.join(input_folder, filename))
        output_path = os.path.join(output_folder, filename)

        if os.path.exists(output_path):
            return 
        if img is not None:
            img = cv2.resize(img, target_size)
            images.append(img)
            cv2.imwrite(output_path, img)

    return images


def create_and_load_dataset(input_folder, output_folder, target_size):
    os.makedirs(output_folder, exist_ok=True)
    images = load_images_from_folder(input_folder, output_folder, target_size)
    
    return images

In [10]:
def process_dataset(dataset_type, output_base_folder, target_size):
    corrected_output = f'{output_base_folder}/{dataset_type}/Corrected'
    normal_output = f'{output_base_folder}/{dataset_type}/Normal'
    reversal_output = f'{output_base_folder}/{dataset_type}/Reversal'

    # Load and save images for each category
    corrected_images = create_and_load_dataset(f'Dyslexia Dataset/{dataset_type}/Corrected', corrected_output, target_size)
    normal_images = create_and_load_dataset(f'Dyslexia Dataset/{dataset_type}/Normal', normal_output, target_size)
    reversal_images = create_and_load_dataset(f'Dyslexia Dataset/{dataset_type}/Reversal', reversal_output, target_size)

    return corrected_images, normal_images, reversal_images

def pick_random_image(directory):
    files = os.listdir(directory)
    
    random_image = random.choice(files)
    image_path = os.path.join(directory, random_image)

    return image_path

In [4]:
sizes = [32, 224, 299]

for s in sizes:
    # Definisi path untuk test set
    test_output_base_folder = f'./Dataset/{s}/Test'
    test_corrected_images, test_normal_images, test_reversal_images = process_dataset('Test', test_output_base_folder, (s,s))

    # Definisi path untuk train set
    train_output_base_folder = f'./Dataset/{s}/Train'
    train_corrected_images, train_normal_images, train_reversal_images = process_dataset('Train', train_output_base_folder, (s,s))

In [15]:
import tensorflow 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
def cnn_arab_model(input_shape, num_classes):
    print(num_classes)
    model = Sequential()

    # Layer 1
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    # Layer 2
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Layer 3
    model.add(Conv2D(16, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    # Layer 4
    model.add(Conv2D(8, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Layer 5
    model.add(Flatten())

    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

def cnn_arab_extraction(model):
    # Now, let's modify the model for feature extraction
    feature_extraction_model = Sequential()

    # Add layers up to the desired feature extraction point
    for layer in model.layers[:-1]:  # Exclude the last layer
        feature_extraction_model.add(layer)

    # Display the modified model summary
    feature_extraction_model.summary()

In [24]:
# Inisialisasi model
input_shape = (224, 224, 3)  # Ganti sesuai dengan dimensi gambar Anda
num_classes = 3  # Ganti sesuai dengan jumlah kelas pada dataset Anda
model = cnn_arab_model(input_shape, num_classes)

# Kompilasi model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Prapemrosesan data dan augmentasi
# Gantilah 'path_to_dataset' dengan path tempat dataset Anda disimpan
# train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
train_generator = train_datagen.flow_from_directory('./Dataset/32/Train/', target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical', classes=['Corrected', 'Normal', 'Reversa;'])
validation_generator = test_datagen.flow_from_directory('./Dataset/32/Test/', target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical', classes=['Corrected', 'Normal', 'Reversa;'])

# Melatih model
epochs = 10  # Ganti sesuai kebutuhan
model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=validation_generator.samples // batch_size)

# Evaluasi model pada set uji
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Test accuracy: {test_acc}')

3
Found 104868 images belonging to 3 classes.
Found 38841 images belonging to 3 classes.
Epoch 1/10
   7/3277 [..............................] - ETA: 5:39:23 - loss: 3.7729 - accuracy: 0.4688

KeyboardInterrupt: 

In [ ]:
# Kompilasi model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Prapemrosesan data dan augmentasi
# Gantilah 'path_to_dataset' dengan path tempat dataset Anda disimpan
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
import pandas as pd

In [8]:
def features_to_dataframe(features, label):
    df = pd.DataFrame(features.reshape(features.shape[0], -1))
    df['Label'] = label
    return df